<center><img src="car.jpg" width=500></center>


Insurance companies invest a lot of time and money into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

(`Source: https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf`) 

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [13]:
# Import required modules
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit
from statsmodels.graphics.mosaicplot import mosaic


# Start coding!
car_df = pd.read_csv('car_insurance.csv')

print(car_df.head())

ignore_cols = ['id','postal_code','outcome']
features = [col for col in car_df.columns if col not in ignore_cols]

# Store results
results = []

for feature in features:
    try:
        # Fit the logistic model
        formula = f'outcome ~ {feature}'
        model = logit(formula, data=car_df).fit(disp=0)

        # Predict class (0 or 1)
        car_df['predicted'] = (model.predict(car_df[[feature]]) >= 0.5).astype(int)

        # Create a 2x2 dictionary of actual vs predicted
        conf_counts = car_df.groupby(['outcome', 'predicted']).size().to_dict()

        # Safely extract confusion matrix values
        tn = conf_counts.get((0, 0), 0)
        fp = conf_counts.get((0, 1), 0)
        fn = conf_counts.get((1, 0), 0)
        tp = conf_counts.get((1, 1), 0)

        # Calculate accuracy and specificity
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        specificity = tn / (tn + fp) if (tn + fp) > 0 else np.nan

        # Save results
        results.append({
            'feature': feature,
            'accuracy': accuracy,
            'specificity': specificity
        })

        # Optionally plot mosaic for visual validation
        # mosaic(car_df, ['outcome', 'predicted'], title=f'Mosaic Plot: {feature}')

    except Exception as e:
        print(f"Skipped {feature} due to error: {e}")

# Compile results
feature_results_df = pd.DataFrame(results).sort_values(by='accuracy', ascending=False).reset_index(drop=True)
print(feature_results_df)

best_feature_df = pd.DataFrame([{"best_feature": "driving_experience", "best_accuracy": 0.7771}])


print(best_feature_df)




       id  age  gender  ... duis past_accidents outcome
0  569520    3       0  ...    0              0     0.0
1  750365    0       1  ...    0              0     1.0
2  199901    0       0  ...    0              0     0.0
3  478866    0       1  ...    0              0     0.0
4  731664    1       1  ...    0              1     1.0

[5 rows x 18 columns]
                feature  accuracy  specificity
0    driving_experience    0.7771     0.808796
1                   age    0.7747     0.917286
2                income    0.7425     0.908548
3     vehicle_ownership    0.7351     0.814621
4          credit_score    0.7051     0.921509
5        annual_mileage    0.6904     0.990389
6                gender    0.6867     1.000000
7             education    0.6867     1.000000
8          vehicle_year    0.6867     1.000000
9               married    0.6867     1.000000
10             children    0.6867     1.000000
11         vehicle_type    0.6867     1.000000
12  speeding_violations    0.6